### [How to return structured data from a model](https://python.langchain.com/docs/how_to/structured_output/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Model

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

#### B. Pydantic

In [8]:
from typing import Optional
from pydantic import BaseModel, Field

class Joke(BaseModel):
    """Joke to tell user"""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None,
        description="How funny the joke is, from 1 to 10"
    )

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Cuentame un chiste de gatos")

Joke(setup='¿Por qué los gatos son malos contadores?', punchline='Porque siempre se les escapan las gatas!', rating=8)

#### C. TypeDict

In [9]:
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
    """Joke to tell user"""
    setup: Annotated[str, ..., "The setup of the joke"]
    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline to the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Cuentame un chiste de gatos")

{'setup': '¿Por qué los gatos son malos contando chistes?',
 'punchline': '¡Porque siempre se quedan en la parte más alta del árbol!',
 'rating': 7}

#### D. JSON

In [10]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}

structured_llm = llm.with_structured_output(json_schema)
structured_llm.invoke("Cuentame un chiste de gatos")

{'setup': '¿Por qué los gatos son malos en las computadoras?',
 'punchline': '¡Porque siempre se sientan sobre el teclado!',
 'rating': 7}

#### E. Multiples esquemas

In [13]:
from typing import Union

class Joke(BaseModel):
    """Joke to tell user"""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None,
        description="How funny the joke is, from 1 to 10"
    )

class ConversationalResponse(BaseModel):
    """Respond in a coneversational manner. Be kind and helpful"""
    response: str = Field(description="A conversational response to the user's query")

class FinalResponse(BaseModel):
    """Final response to user"""
    response: Union[Joke, ConversationalResponse]

structured_llm = llm.with_structured_output(FinalResponse)
print(structured_llm.invoke("Cuentame un chiste de gatos"))
print(structured_llm.invoke("Como estás hoy?"))

response=Joke(setup='¿Qué hace un gato cuando cae al agua?', punchline='¡Nada!', rating=7)
response=ConversationalResponse(response='¡Estoy aquí y listo para ayudarte! ¿Y tú, cómo estás hoy?')


In [14]:
from typing import Optional, Union
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
    """Joke to tell user"""
    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline to the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

class ConversationalResponse(TypedDict):
    """Respond in a coneversational manner. Be kind and helpful"""
    response: Annotated[str, ..., "A conversational response to the user's query"]

class FinalResponse(TypedDict):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)
print(structured_llm.invoke("Cuentame un chiste de gatos"))
print(structured_llm.invoke("Como estás hoy?"))

{'final_output': {'setup': '¿Por qué los gatos no juegan a las cartas?', 'punchline': 'Porque siempre están preocupados por el maullido.', 'rating': 7}}
{'final_output': {'response': '¡Hola! Estoy aquí y listo para ayudarte. ¿Y tú, cómo estás hoy?'}}


#### F. Streaming

In [15]:
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
    """Joke to tell user"""
    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline to the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)

for chunk in structured_llm.stream("Cuentame un chiste de gatos"):
    print(chunk)

{}
{'setup': ''}
{'setup': '¿'}
{'setup': '¿Por'}
{'setup': '¿Por qué'}
{'setup': '¿Por qué los'}
{'setup': '¿Por qué los gatos'}
{'setup': '¿Por qué los gatos son'}
{'setup': '¿Por qué los gatos son tan'}
{'setup': '¿Por qué los gatos son tan buenos'}
{'setup': '¿Por qué los gatos son tan buenos en'}
{'setup': '¿Por qué los gatos son tan buenos en la'}
{'setup': '¿Por qué los gatos son tan buenos en la computadora'}
{'setup': '¿Por qué los gatos son tan buenos en la computadora?'}
{'setup': '¿Por qué los gatos son tan buenos en la computadora?', 'punchline': ''}
{'setup': '¿Por qué los gatos son tan buenos en la computadora?', 'punchline': 'Porque'}
{'setup': '¿Por qué los gatos son tan buenos en la computadora?', 'punchline': 'Porque siempre'}
{'setup': '¿Por qué los gatos son tan buenos en la computadora?', 'punchline': 'Porque siempre están'}
{'setup': '¿Por qué los gatos son tan buenos en la computadora?', 'punchline': 'Porque siempre están buscando'}
{'setup': '¿Por qué los gatos

#### G. Few-shot prompting

In [17]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

{'setup': 'Woodpecker',
 'punchline': "Woodpecker knocked on my door, but I told it I'm already 'pecked' out from all this knocking!",
 'rating': 7}

In [18]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

{'setup': 'Crocodile',
 'punchline': "Crocodile tears are real, but I prefer my jokes to be a little more 'reptile'!",
 'rating': 6}